In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import re
import statsmodels.api as sm

#Fonction pour trouver les codes correspondants à un pays entre les codes de pays et le nom des pays :"
def refresh_dico(Name,Code,df):
    dic = {}
    for i in df[[Name,Code]].dropna().drop_duplicates().iterrows() :
        dic[i[1][Name]] = i[1][Code]
    return (dic)
#fonction pour trouver toutes les lignes qui correspondent à un pays où à une variable en particulier

def rech_ligne_pays (y,df) :
    expression = re.compile(dic_pays[y]+".*?;")
    return(list(i[:-1] for i in (expression.findall(';'.join(df.index)+';'))))

def rech_ligne_indic (y,df):
    expression = re.compile(".{,4}"+y+";")
    return(list(i[:-1] for i in (expression.findall(';'.join(df.index)+';'))))

def graph_regress(regresseur,regresse) :
    sns.lmplot(x=regresse, y=regresseur, data=df1.drop(['Country Name','Country Code','Indicator Name'], axis = 1).loc[[regresse,regresseur]].dropna(axis = 1).T, robust = True)

def regress (regresseur,regresse) :
    x = sm.add_constant(df1.drop(['Country Name','Country Code','Indicator Name'],axis =1).loc[regresseur].to_numpy())
    y = df1.drop(['Country Name','Country Code','Indicator Name'],axis=1).loc[regresse].to_numpy()

    model = sm.OLS(y,x,'drop')
    results = model.fit()
    return(results)

def create_gdf (df_geo,map_monde) :
    map_monde_clean = pd.DataFrame()
    map_monde_clean[['Country Code','geometry']] = map_monde[['iso_a3','geometry']].dropna()
    gdf = gpd.GeoDataFrame(df_geo.dropna().reset_index().merge(map_monde_clean,on = 'Country Code'),
                           geometry=df_geo.dropna()
                                       .reset_index()
                                       .merge(map_monde_clean,on = 'Country Code')['geometry'])
    gdf = gdf.reset_index()
    gdf = gdf.set_index('Pays_indic')
    return(gdf)

def log_norm (df_norm) :
    return(np.log10(df_norm.astype(np.float64))/np.log10(df_norm.astype(np.float64).max()))

def geo_plot (df,indic,annee,couleur = 'Oranges',norm=True) : 
    #On importe la map monde dans un data frame avec ses données géographiques : 
    map_monde = gpd.read_file('map_monde (1).json')
    #on créé deux dictionnaires différents pour pouvoir adapter les codes pays et merge le plus correctemment possible
    #les deux dataframe: 
    dic_map = refresh_dico('iso_a3','sovereignt',map_monde)
    dic_code = refresh_dico('Country Code','Country Name',df)  
    liste_non_concord = []
    
    #On récupère tous les pays pour lesquels les codes pays ne correspondent pas :
    for i in dic_map.keys():
        j = ''
        try :
            j = dic_code[i]
        except KeyError:
            liste_non_concord.append(dic_map[i])

    for i in liste_non_concord:
        try :
            map_monde.loc[map_monde['sovereignt'] == i,'iso_a3'] = dic_pays[i]
        except KeyError:
            print(i +' n\'a aucun équivalent dans la table des données')
            
    #On peut dés maintenant commencer le plot :
    ax = gplt.polyplot(map_monde['geometry'],
                       edgecolor='None',
                       facecolor='lightgray',
                       figsize=(18, 7))

    gdf = create_gdf(df.loc[rech_ligne_indic(indic,df),['Country Code',annee]],map_monde)
    if norm :
        gdf[annee] = log_norm(gdf[annee])
    gplt.choropleth(gdf,
                hue = annee,
                cmap = couleur,
                #norm=colors.LogNorm(vmin=gdf[annee].min(),
                #                    vmax=gdf[annee].max()),
                legend=True,
                ax=ax)
    plt.show()
#/!\ il faut entre le nom avec des caractères regex pour les ( et %)
        

On construit un dictionnaire nom du pays / code du pays :

# On va télécharger les bases de données et repérer les variables pertinentes dans chacune des bases

### On commmence par créer les dataframe

Il faut légèrement modifier le fichier csv dézippé pour enlever les deux premières lignes, qui ne comprennent que des infos générales sur le format du fichier mais qui entrainaient un problème d'ouverture via la commande pd.read_csv car elles comportaient des virgules.

On peut ensuite ouvrir sereinement toutes les base modifiées ainsi.

In [2]:
agri_forest = pd.read_csv("chiffre_agri_foret_peche.csv")
augment_pib = pd.read_csv("chiffre_augment_pib.csv")
energie_utilisee = pd.read_csv("chiffre_energie_utilise.csv")
emission_GES = pd.read_csv("chiffre_GES_total.csv")
nucle_total = pd.read_csv('chiffre_nucle_total_nuke_total_fossile.csv')
part_indus_construc = pd.read_csv('chiffre_part_indus_construc.csv')
pib = pd.read_csv("chiffre_pib.csv")
nom_pays = pib['Country Name']


In [3]:
df_list  = [agri_forest, augment_pib, energie_utilisee, emission_GES, nucle_total, part_indus_construc, pib]

### On regarde chaque dataframe pour identifier les variables pertinentes

In [4]:
agri_forest.head(1)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Aruba,ABW,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On observe que tous les df sont construits de la même manière. On a le nom du pays, son abréviation, le nom de l'indicateur utilisé, son code, puis sa valeur pour chaque année. On a aussi toujours une colonne vide, qui ne nous gène pas vraiment.

### On nettoie les bases en enlevant les informations qui ne nous intéresse pas

Il faudra enlever la colomne "unnamed", les colomnes qui correpondent aux années qui ne nous intéresssent pas, et éventuellement les lignes (=pays ou région) que l'on ne garde pas.

Par exemple, conserve-t-on les pays comme Aruba dont on ne connait que le PIB? Je pense que la question elle est vite répondue.

Pareil, est-ce que ça a vraiement un intérêt de garder les zones géographiques du type Asie du Sud-Est, etc?  ce ne sont que des aggrégats de pays, donc pas forcément.

### On fusionne les dataframe pour aggréger tous ces indicateurs

On peut envisager plusieurs manières de fusionner ces df. Soit on a en index Pays_Année et les colomnes correspindent chacune à un indice, soit on a Pays_Indicateur et les colonnes correspondent chacune a une année. Ce sont les formats wide et long (cf tp2).

Pour des raisons de lisibilité, il est évident que l'on va conserver le nom de l'indicateur, et non pas son nom de code. En revanche, il faudra voir à l'usage si on préfère utiler le nom complet des pays ou seulement leur nom de code

On concatène les df, on les trie et on met en index la variable créée à cet effet ci-dessus.

In [5]:
#On fait ici la modif sur tous les df à l'aide d'une boucle for, 
for df in df_list :
    df['Pays_indic'] = df['Country Code'].str[:3] + '_' + df['Indicator Name'].str[:99]
#part_indus_construc['Pays_indic']

df = pd.concat(df_list)
df = df.sort_values('Pays_indic').set_index('Pays_indic')
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
Pays_indic,,,,,,,,,,,,,,,,,,,,,
"ABW_Agriculture, forestry, and fishing, value added (% of GDP)",Aruba,ABW,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABW_Alternative and nuclear energy (% of total energy use),Aruba,ABW,Alternative and nuclear energy (% of total ene...,EG.USE.COMM.CL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABW_Energy use (kg of oil equivalent per capita),Aruba,ABW,Energy use (kg of oil equivalent per capita),EG.USE.PCAP.KG.OE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABW_GDP (current US$),Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.534637e+09,2.701676e+09,2.765363e+09,2.919553e+09,2.965922e+09,3.056425e+09,NaN,NaN,NaN,NaN
ABW_GDP growth (annual %),Aruba,ABW,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.369863e+00,4.198232e+00,3.000000e-01,5.700001e+00,2.100000e+00,1.999999e+00,NaN,NaN,NaN,NaN


### Suppression des colonnes et lignes inutiles

On a ici la liste des colonnes que l'on supprimme pour se restreindre aux données qui nous intéressent, sutrement dit à partir de 1980 (je ne sais plsu si on avait dit 1980 ou 1985).

In [6]:
useless_data_list = ['Unnamed: 65', 'Indicator Code', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', 
                   '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '2020']
df = df.drop(useless_data_list, axis = 1)
#df.sample(15)

On va chercher à enlever les lignes qui ne contiennent que peu d'infos (au moins 10 valeurs manquantes)

In [7]:
df1 = df
nan_dict = {}
for i in range(len(df1.index)) :
    nb_nan = df1.iloc[i].isnull().sum()
    if nb_nan >10 :
        nan_dict[list(df.index)[i]] = nb_nan


df1 = df1.drop(index = nan_dict.keys(), axis = 0)
df1.shape

(1259, 43)

En faisant cela, on supprme environ 600 lignes. Pour le rafiner, il faudrait voir si l'on ne doit pas supprimer toutes les données d'un pays lorsque l'on a que des infos sur deux ou trois indicateurs (notamment qi que sur le PIB). On doit pouvoir s'en sortir en supprimant tous les indexs qui commencent par le code du pays détecté de cette manière.

[Adrien]La fonction pour récupérer les indexs des pays où on a moins de 3 indicateurs.

In [8]:
dic_pays = refresh_dico()

TypeError: refresh_dico() missing 3 required positional arguments: 'Name', 'Code', and 'df'

In [ ]:
for i in dic_pays.keys() :
    if len(rech_ligne_pays(i,df1))<3 :
        df1 = df1.drop(rech_ligne_pays(i,df1))
dic_pays = refresh_dico()
#ancienne fonction que j'ai écrite plus tot (beaucoup plus lente) :

#for i in nom_pays : 
#    compter_nb_indic = df1.apply(lambda x: True if x['Country Name'] == i else False, axis = 1)
#    if len(compter_nb_indic[compter_nb_indic == True].index) <3 :
#        liste_pays_dic[i] = compter_nb_indic[compter_nb_indic == True].index'''


On supprime les lignes des pays dont on a trop peu d'indicateurs complet : (servait pourl'ancienne fonction)

In [ ]:
#for i in liste_pays_dic.keys():
#    df1 = df1.drop(index = liste_pays_dic[i], axis = 1)

On a supprimer en tout 58 lignes. De plus on va aussi supprimer de la base les aggrégats de pays (ces lignes là sont colinéaires aux autres, ce qui risque de nous poser problème plus tard dans la prédiction). Voilà les régions à supprimer : Upper middle income, Latin America & the Caribbean (IDA & IBRD countries),Middle East & North Africa (IDA & IBRD countries),South Asia (IDA & IBRD), Middle East & North Africa (IDA & IBRD countries), Latin America & the Caribbean (IDA & IBRD countries),East Asia & Pacific (IDA & IBRD countries), Europe & Central Asia (IDA & IBRD countries), Sub-Saharan Africa, Sub-Saharan Africa (excluding high income), Post-demographic dividend, Pre-demographic dividend, OECD members, North America, Middle East & North Africa (excluding high income), Middle East & North Africa,Late-demographic dividend, Lower middle income
Low & middle income,Latin America & Caribbean,Least developed countries: UN classification, Latin America & Caribbean (excluding high income),IBRD only,IDA & IBRD total,IDA total,IDA blend,Heavily indebted poor countries (HIPC),High income, European Union, Fragile and conflict affected situations,East Asia & Pacific (excluding high income)
Early-demographic dividend, East Asia & Pacific, Europe & Central Asia (excluding high income), Europe & Central Asia,Central Europe and the Baltics


à propos des aggrégats (comment ils aggrègent :)

Aggregation Rules

Aggregates are based on the World Bank’s regional and income classification of economies. Because of missing data, aggregates for groups of economies should be treated as approximations of unknown totals or average values. Regional and income group aggregates are based on the largest available set of data. The aggregation rules are intended to yield estimates for a consistent set of economies from one period to the next and for all indicators. Small differences between sums of subgroup aggregates and overall totals and averages may occur because of the approximations used. In addition, compilation errors and data reporting practices may cause discrepancies in theoretically identical aggregates such as world exports and world imports.

Five methods of aggregation are used in the World Development Indicators:

For group and world totals denoted in the tables by a t, missing data are imputed based on the relationship of the sum of available data to the total in the year of the previous estimate. The imputation process works forward and backward from 2010. Missing values in 2010 are imputed using one of several proxy variables for which complete data are available in that year. The imputed value is calculated so that it (or its proxy) bears the same relationship to the total of available data. Imputed values are usually not calculated if missing data account for more than a third of the total in the benchmark year. The variables used as proxies are GNI in U.S. dollars, total population, exports and imports of goods and services in U.S. dollars, and value added in agriculture, industry, manufacturing, and services in U.S. dollars.

Aggregates marked by an s are sums of available data. Missing values are not imputed. Sums are not computed if more than a third of the observations in the series or a proxy for the series are missing in a given year.

Aggregates of ratios are generally calculated as weighted averages of the ratios (indicated by w) using the value of the denominator or, in some cases, another indicator as a weight. The aggregate ratios are based on available data, including data for economies not shown in the main tables. Missing values are assumed to have the same average value as the available data. No aggregate is calculated if missing data account for more than a third of the value of weights in the benchmark year. In a few cases the aggregate ratio may be computed as the ratio of group totals after imputing values for missing data according to the above rules for computing totals.

Aggregate growth rates are generally calculated as a weighted average of growth rates (and indicated by a w). In a few cases growth rates may be computed from time series of group totals. Growth rates are not calculated if more than half the observations in a period are missing. For further discussion of methods of computing growth rates see below.

Aggregates denoted by an m are medians of the values shown in the table. No value is shown if more than half the observations for countries with a population of more than 1 million are missing. Exceptions to the rules occur throughout the book. Depending on the judgment of World Bank analysts, the aggregates may be based on as little as 50 percent of the available data.
In other cases, where missing or excluded values are judged to be small or irrelevant, aggregates are based only on the data shown in the tables.

Du coup en fonction de comment ils ont agrégés les données ça peut être intéressant ou non de les conserver. Notamment ceux pour lesquels ils ont utilisés la 1ère méthode puisqu'elle nous permet d'avoir une approximation des données qui nous manque (après est ce qu'on veut garder des approximations ? Dans tous les cas il faudra voir je pense si nos prédictions sont robustes à leur ajout ou non)

In [ ]:
#/!\ il ne faut faire tourner qu'une seule fois cette fonction, sinon on a une KeyError
liste_supression = ["Upper middle income", "Latin America & the Caribbean (IDA & IBRD countries)","Middle East & North Africa (IDA & IBRD countries)","South Asia (IDA & IBRD)","Middle East & North Africa (IDA & IBRD countries)", "Latin America & the Caribbean (IDA & IBRD countries)","East Asia & Pacific (IDA & IBRD countries)","Europe & Central Asia (IDA & IBRD countries)","Sub-Saharan Africa", "Sub-Saharan Africa (excluding high income)","Post-demographic dividend", "Pre-demographic dividend", "OECD members", "North America", "Middle East & North Africa (excluding high income)", "Middle East & North Africa","Late-demographic dividend", "Lower middle income","Low & middle income","Latin America & Caribbean","Least developed countries: UN classification", "Latin America & Caribbean (excluding high income)","IBRD only","IDA & IBRD total","IDA total","IDA blend","Heavily indebted poor countries (HIPC)","High income","European Union", "Fragile and conflict affected situations","East Asia & Pacific (excluding high income)","Early-demographic dividend","East Asia & Pacific","Europe & Central Asia (excluding high income)","Europe & Central Asia","Central Europe and the Baltics"]

for i in liste_supression :
    df1 = df1.drop(labels = rech_ligne_pays(i,df1)) 
dic_pays = refresh_dico()


On a supprimer en tout une centaine de lignes encore. On actualise la liste pour stocker uniquement les noms des pays restants :

# Premières représentations graphiques

On créé un vecteur temps pour pouvoir créer des graphiques en fonction du temps facilement.

In [ ]:
t = np.linspace(1980,2019,40)

In [ ]:
#Sur deux subplots différents
plt.subplot(211)
plt.plot(t[0:31],df1.loc['FRA_Émissions totales de GES (kt d’équivalent CO2)'][2:42].to_numpy()[1:32].T)
plt.subplot(212)
plt.plot(t[0:31],df1.loc['FRA_GDP growth (annual %)'][2:42].to_numpy()[1:32].T)

plt.show()

#Sur le même plot\n",

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Emission de GES', color=color)
ax1.plot(t[0:31], df1.loc['FRA_Émissions totales de GES (kt d’équivalent CO2)'][2:42].to_numpy()[1:32].T, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  #  commence un deuxième axe qui partage le même axe x

color = 'tab:blue'
ax2.set_ylabel('PIB', color=color)  
ax2.plot(t[0:31], df1.loc['FRA_GDP (current US$)'][2:42].to_numpy()[1:32].T, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
#Sur deux subplots différents
plt.subplot(211)
plt.plot(t[0:31],df1.loc['USA_Émissions totales de GES (kt d’équivalent CO2)'][2:42].to_numpy()[1:32].T)
plt.subplot(212)
plt.plot(t[0:31],df1.loc['USA_GDP growth (annual %)'][2:42].to_numpy()[1:32].T)

plt.show()

#Sur le même plot\n",

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Emission de GES', color=color)
ax1.plot(t[0:31], df1.loc['USA_Émissions totales de GES (kt d’équivalent CO2)'][2:42].to_numpy()[1:32].T, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  #  commence un deuxième axe qui partage le même axe x

color = 'tab:blue'
ax2.set_ylabel('PIB', color=color)  
ax2.plot(t[0:31], df1.loc['USA_GDP (current US$)'][2:42].to_numpy()[1:32].T, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()


On va créer un nouvel indicateur qui va nous donner la croissance à l'année des gazs à effet de serre pour chaque pays :

In [ ]:
for i in rech_ligne_indic('Émissions totales de GES \(kt d’équivalent CO2\)',df1) :
    df1.loc[i[0:3]+'_croissance émissions GES (kt d’équivalent CO2)'] = df1.loc[i][3:44].astype(np.float64).diff()
    df1.loc[i[0:3]+'_croissance émissions GES (pourcent)'] = df1.loc[i[0:3]+'_croissance émissions GES (kt d’équivalent CO2)']*100 /(df1.loc[i[0:3]+'_Émissions totales de GES (kt d’équivalent CO2)']-df1.loc[i[0:3]+'_croissance émissions GES (kt d’équivalent CO2)'] ) 

In [ ]:
rech_ligne_indic('Émissions totales de GES \(kt d’équivalent CO2\)',df1)

Maintenant qu'on à la croissance des émissions de gazs à effet de serre par an, on peut représenter la croissance du PIB et celle-ci sur le même graphe :

In [ ]:
#Sur le même plot,

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Emission de GES', color=color)
ax1.plot(t[0:31], df1.loc['USA_croissance émissions GES (pourcent)'][2:42].to_numpy()[1:32].T, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  #  commence un deuxième axe qui partage le même axe x

color = 'tab:blue'
ax2.set_ylabel('Croissance du PIB', color=color)  
ax2.plot(t[0:31], df1.loc['USA_GDP growth (annual %)'][2:42].to_numpy()[1:32].T, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

On peut voir qu'il y a apparemment une certaine concommitance des courbes, cela-dit il faut faire attention aux effets d'échelles car ici le PIB ne passe presque jamais dans le négatif (ce qui est loin d'être le cas de la croissance des émissions de gaz à effet de serre). On va essayer de le faire dans le cas de la France pour se faire plus d'idées :

In [ ]:
#Sur le même plot,

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('Emission de GES', color=color)
ax1.plot(t[0:31], df1.loc['FRA_croissance émissions GES (pourcent)'][2:42].to_numpy()[1:32].T, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  #  commence un deuxième axe qui partage le même axe x

color = 'tab:blue'
ax2.set_ylabel('Croissance du PIB', color=color)  
ax2.plot(t[0:31], df1.loc['FRA_GDP growth (annual %)'][2:42].to_numpy()[1:32].T, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

Idée : on pourrait essayer de faire du clustering afin de déterminer la dépendance de la croissance à certains facteurs afin d'essayer de prédire la dépendance de la croissance à l'émission de gaz à effet de serre.

Je vais essayer de régresser le PIB sur l'émission de gaz à effet de serre dans plusieurs pays : la France, Les Etats Unis, La Chine et le Zimbabwe

## Régressions

### Régressions avec scipy

In [ ]:
import scipy.optimize
def affin(x,a,b): 
    return a*x + b

def regress2 (country_code, regresse, regresseur, courbe = False):
    
    interm= pd.DataFrame(df1.loc[[country_code + regresse, country_code + regresseur]]).dropna(axis = 1)
    if interm.shape != (2,0): 
        
        regresse_val = pd.DataFrame(interm.loc[country_code + regresse][3:30])[country_code + regresse].to_numpy(dtype = float)
        regresseur_val = pd.DataFrame(interm.loc[country_code + regresseur][3:30])[country_code + regresseur].to_numpy(dtype = float)

        popt, pcov = scipy.optimize.curve_fit(affin, regresseur_val, regresse_val)
        coeff, cst = popt[0], popt[1]

        if courbe == True :
            fig, ax1 = plt.subplots()

            color = 'tab:red'
            ax1.set_xlabel('time (s)' + country_code, color = 'k')
            ax1.set_ylabel(regresse, color='k')
            ax1.plot(t[0:31], df1.loc[country_code + regresse][2:42].to_numpy()[1:32].T, color='k')
            ax1.tick_params(axis='y', labelcolor='k')

            ax1.plot(t[0:31], coeff*df1.loc[country_code + regresseur][2:42].to_numpy()[1:32].T + cst, color='b')

            fig.tight_layout()  # otherwise the right y-label is slightly clipped
            plt.show()

        return coeff, cst
    else : return 
    
code_list = []
for i in df1['Country Code']:
    if i not in code_list : 
        code_list.append(i)
code_list.pop(-1)

regresseur = '_GDP (current US$)'
regresse = '_Émissions totales de GES (kt d’équivalent CO2)'
dic = {}

for code in code_list:
    dic[code] = regress2(code, regresse, regresseur, False)
    if dic[code] == None:
        dic.pop(code)
#Attention, il faudra donner un nom explicit à chaque df, différent pour chaque régression.
regress2_df = pd.DataFrame(dic.values(), dic.keys(), ['Coeff', 'Cst'])
regress2_df.head()

### Régressions graphiques avec Seaborn

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics

In [ ]:

for i in ['IND','CHN','ZWE','SWE','USA']:
    sns.lmplot(x=i+"_GDP growth (annual %)", y=i+"_croissance émissions GES (pourcent)", 
               data=df1.drop(['Country Name','Country Code','Indicator Name'], axis = 1).loc[[i+'_GDP growth (annual %)',i+'_croissance émissions GES (pourcent)']].dropna(axis = 1).T, robust = True)
    
def graph_regress(regresseur,regresse) :
    sns.lmplot(x=regresse, y=regresseur, data=df1.drop(['Country Name',
        'Country Code','Indicator Name'], axis = 1).loc[[regresse,regresseur]].dropna(axis = 1).T, robust = True)

### Régressions avec statsmodels

In [ ]:
#On a ici la syntaxe pour faire une régression avec statsmodels, ainsi que la forme de la sortie
x = sm.add_constant(df1.drop(['Country Name','Country Code','Indicator Name'],axis =1).loc['USA_Émissions totales de GES (kt d’équivalent CO2)'].to_numpy())
y = df1.drop(['Country Name','Country Code','Indicator Name'],axis=1).loc['USA_GDP (current US$)'].to_numpy()

model = sm.OLS(y,x,'drop')
results = model.fit()
print(results.summary())


In [ ]:
#On voit ici coment récupérer les coefficients voulus
print(dir(results))
results._get_robustcov_results
results.rsquared

On définit ici la fonction qui permet d'automatiser les régressions sur tous les pays, puis on garde les résultats dans un dataframe.

In [ ]:
def regress (country_code, regresseur, regresse) :
    
    if country_code + regresse in df1.index and country_code + regresseur in df1.index :
        interm= pd.DataFrame(df1.loc[[country_code + regresse, country_code + regresseur]]).drop(['Country Name',
        'Country Code','Indicator Name'],axis =1).dropna(axis = 1)

        if interm.shape != (2,0): 
            x = sm.add_constant(interm.loc[country_code + regresseur].to_numpy())
            y = interm.loc[country_code + regresse].to_numpy()

            model = sm.OLS(y,x,'drop')
            results = model.fit()

            r2 = results.rsquared
            cst = results.params[0]
            coef = results.params[1]

            return cst, coef, r2
    
    else : return

    
    
code_list = []
for i in df1['Country Code']:
    if i not in code_list : 
        code_list.append(i)
code_list.pop(-1)

regresse = '_GDP (current US$)'
regresseur = '_Émissions totales de GES (kt d’équivalent CO2)'

for code in code_list:
    dic[code] = regress(code, regresse, regresseur)
    if dic[code] == None:
        dic.pop(code)
        
regress_df = pd.DataFrame(dic.values(), dic.keys(), ['Constante', 'Coefficient', 'R2'])
regress_df.head()

On peut ensuite effectuer toutes les régressions que l'on souhaite graâce à cette fonction, en changeant simplementles variables regresseur et regresse. Il suffit à chaque fois de créer un dataframe avec un nom différent, de donner des noms plus explicits aux colonnes, et de les concaténer après.

Exemple ci-dessous, en concaténant la régression de gdp sur ges avec celle sur leurs taux de variation

In [ ]:
regresse = '_GDP (current US$)'
regresseur = '_Émissions totales de GES (kt d’équivalent CO2)'


for code in code_list:
    dic[code] = regress(code, regresse, regresseur)
    if dic[code] == None:
        dic.pop(code)
        
regress1_df = pd.DataFrame(dic.values(), dic.keys(), ['Constante GDP sur total GES',
                                                     'Coefficient GDP sur total GES', 'R2 GDP sur total GES'])
regress1_df.head()




regresse = '_GDP growth (annual %)'
regresseur = '_croissance émissions GES (kt d’équivalent CO2)'

dic = {}
for code in code_list:
    dic[code] = regress(code, regresse, regresseur)
    if dic[code] == None:
        dic.pop(code)

regress2_df = pd.DataFrame(dic.values(), dic.keys(), ['Constante GDP_var sur GES_var', 'Coefficient GDP_var sur GES_var', 'R2 GDP_var sur GES_var'])

regress_df = pd.concat([regress1_df, regress2_df], axis = 1)
regress_df.head()

In [ ]:
regress_df.plot(x='Coefficient GDP sur total GES', y='Coefficient GDP_var sur GES_var', kind = 'scatter', label = "Ce graphique n'est pas extraordinaire")